In [1]:
%load_ext autoreload
%autoreload 2

import random
import pandas
from typing import Dict, List, Tuple, Set
from problem import *
from round import *
from heuristics import *
from utils import *

In [ ]:
%reset

## DATA FOR CVILLE ##

In [ ]:
#usa_va_charlottesville_city_adult_activity_location_assignment_week.csv
adult_df = pandas.read_csv("usa_va_charlottesville_city_adult_activity_location_assignment_week.csv")
adult_df

In [ ]:
HOME_SHIFT=1000000000
adult_df.loc[adult_df.activity_type == 1, "lid"] += HOME_SHIFT
adult_df

In [ ]:
df = adult_df.groupby(['lid'])['lid'].count().reset_index(name = 'Count').sort_values(['Count'], ascending=False)
df

In [ ]:
print(adult_df[adult_df["pid"]==5627703])

## Distance Function ##

In [ ]:
import geopy.distance

coords_1 = (50, 20)
coords_2 = (51, 20)

print(geopy.distance.distance(coords_1, coords_2).km)

In [ ]:
#usa_va_charlottesville_city_adult_activity_location_assignment_week.csv
df = pandas.read_csv("usa_va_charlottesville_city_adult_activity_location_assignment_week.csv")
HOME_SHIFT=1000000000
df.loc[df.activity_type == 1, "lid"] += HOME_SHIFT

key = {}

"""
Read in activity locations file
"""
location_file_act = open("usa_va_charlottesville_city_activity_locations.csv", "r")
lines_act = location_file_act.readlines()
print(lines_act[0])
lines_act = location_file_act.readlines()[1:]

"""
Read in residence locations file
"""
location_file_res = open("usa_va_charlottesville_city_residence_locations.csv", "r")
lines_res = location_file_res.readlines()[1:]
for line in lines_res:
    line[0] += HOME_SHIFT

lines = lines_act + lines_res

count = 0
G = []
for line in lines:
    
    key[count] = (line[0], line[1], line[2])
    count += 1
    
    G.append([])
    for i in range(count):
        coords_1 = (line[2], line[1])
        coords_2 = (key[i][2], key[i][1])          #Make into namedtuple
        G[-1].append(geopy.distance.distance(coords_1, coords_2).km)

'''G = []
for index, row in df.iterrows():
    #print(row)
    G.append([])
    for prev_ind, prev_row in df[:index].iterrows():
        #print(prev)
        coords_1 = (row['latitude'], row['longitude'])
        coords_2 = (prev_row['latitude'], prev_row['longitude'])
        G[-1].append(geopy.distance.distance(coords_1, coords_2).km)
    G[-1].append(0)'''

for row in G:
    print(row)

In [ ]:
df_activity = pandas.read_csv("usa_va_charlottesville_city_activity_locations.csv").rename({"alid": "lid"}, axis = 'columns')
print(len(df_activity))
df_residence = pandas.read_csv("usa_va_charlottesville_city_residence_locations.csv")
HOME_SHIFT=1000000000
df_residence['rlid'] = df_residence['rlid'].apply(lambda x: x+HOME_SHIFT)
df_residence = df_residence.rename({"rlid": "lid"}, axis = 'columns')

df_locations = pandas.concat([df_activity[['lid', 'longitude', 'latitude']], df_residence[['lid', 'longitude', 'latitude']]])
df_locations

In [ ]:
import json

G = []
location_indexing = {}
for index, row in df_locations.iterrows:
    location_indexing[row['lid']] = index
    G.append([])
    for prev_index, prev_row in df_locations[:index].iterrows():
        coords_1 = (row['latitude'], row['longitude'])
        coords_2 = (prev_row['latitude'], prev_row['longitude'])          #Make into namedtuple
        G[-1].append(geopy.distance.distance(coords_1, coords_2).km)
    G[-1].append(0)

with open('cville_location_G.json', 'w') as f:
    json.dump(G, f)

In [ ]:
df = pandas.read_csv("usa_va_charlottesville_city_adult_activity_location_assignment_week.csv")
HOME_SHIFT=1000000000
df.loc[df.activity_type == 1, "lid"] += HOME_SHIFT

client_indexing = {}
client_locations = []
for index, row in df.iterrows():
    if row['pid'] in client_indexing:
        if row['activity_type'] == 1:
            client_locations[index].insert(0, location_indexing[row['lid']])
        else:
            client_locations[index].append(location_indexing[row['lid']])
    else:
        client_indexing[row['pid']] = index
        client_locations.append([location_indexing[row['lid']]])

with open('cville_client_locations.json', 'w') as f:
    json.dump(client_locations, f)

with open('cville_indexing.json', 'w') as f:
    json.dump(location_indexing, f)
    json.dump(client_indexing, f)

In [ ]:
!pip install haversine

In [ ]:
import haversine as hs
loc1=(28.426846,77.088834)
loc2=(28.394231,77.050308)

coords_1 = (52.2296756, 21.0122287)
coords_2 = (52.406374, 16.9251681)

hs.haversine(coords_1, coords_2)

## DATA ##

In [ ]:
#HOW MUCH A SET OF LOCATIONS CAN COVER

#locations = pandas.read_csv("usa_va_charlottesville_city_activity_locations.csv").rename({"alid": "lid"}, axis = 'columns')
#locations['activity'] = locations['work'] + locations['shopping'] + locations['school'] + locations['other'] + locations['college'] + locations['religion']
#locations = locations.sort_values(by="activity", ascending=False)
#locations = locations.reset_index(drop=True)
#print(locations)


assignments = pandas.read_csv("usa_va_charlottesville_city_adult_activity_location_assignment_week.csv")
HOME_SHIFT=1000000000
assignments.loc[assignments.activity_type == 1, "lid"] += HOME_SHIFT

total_clients = len(assignments.groupby(['pid']))

assignments = assignments.groupby(['lid'])['pid'].apply(set).reset_index(name = 'pid')

#assignments = assignments.set_index('lid')
#total_homes = len(assignments.groupby(['hid']))
#assignments = assignments.groupby(['lid'])['hid'].apply(set).reset_index(name = 'hid')

locations = assignments.copy()
locations['activity'] = locations['pid'].apply(lambda x: len(x))
locations = locations.sort_values(by = 'activity', ascending = False)

assignments = assignments.set_index('lid')

print(locations.head())

In [ ]:
#number_selected = 120

plot_x = []
plot_y = []

for i in range(10, 32, 2):
    number_selected = i
    covered = set()

    #print(locations[:number_selected].lid)
    for loc in locations[:number_selected].lid:
        if loc in assignments.index:
            covered = covered.union(assignments.loc[loc].at['pid'])
    
    print(i, len(covered)/total_clients)
    
    #plot_x.append(i)
    #plot_y.append(len(covered)/total_homes)

#plt.plot(plot_x, plot_y)

In [ ]:
import matplotlib.pyplot as plt

assignments = pandas.read_csv("usa_va_charlottesville_city_adult_activity_location_assignment_week.csv")
HOME_SHIFT=1000000000
assignments.loc[assignments.activity_type == 1, "lid"] += HOME_SHIFT

#total_homes = len(assignments.groupby(['pid']))

assignments = assignments.groupby(['hid'])['lid'].apply(set)
#print(assignments)
#.reset_index(name = 'pid')
#assignments = assignments.set_index('pid')

plt.hist([len(l) for hid, l in assignments.items()], bins = 50)
plt.plot()

In [ ]:
locations = pandas.read_csv("usa_va_charlottesville_city_activity_locations.csv").rename({"alid": "lid"}, axis = 'columns')
locations['activity'] = locations['work'] + locations['shopping'] + locations['school'] + locations['other'] + locations['college'] + locations['religion']
locations = locations.sort_values(by="activity", ascending=False)
locations = locations.reset_index(drop=True)
print(len(locations))

## Formating Data ##

In [ ]:
df_activity = pandas.read_csv("usa_va_charlottesville_city_activity_locations.csv").rename({"alid": "lid"}, axis = 'columns')
df_residence = pandas.read_csv("usa_va_charlottesville_city_residence_locations.csv")

HOME_SHIFT=1000000000
df_residence['rlid'] = df_residence['rlid'].apply(lambda x: x+HOME_SHIFT)
df_residence = df_residence.rename({"rlid": "lid"}, axis = 'columns')

df_locations = pandas.concat([df_activity[['lid', 'longitude', 'latitude']], df_residence[['lid', 'longitude', 'latitude']]]).reset_index(drop = True)
df_locations
#print(df_locations[df_locations['lid'] == 1229162])

location_dict = df_locations[['lid']]
location_dict.reset_index(inplace=True)
location_dict = location_dict.set_index('lid')

In [ ]:
import geopy.distance

def calculate_distance(loc1, loc2):
    print(df_locations.head())
    
    coord1 = (df_locations.loc[loc1, 'latitude'], df_locations.loc[loc1, 'longitude'])
    coord2 = (df_locations.loc[loc2, 'latitude'], df_locations.loc[loc2, 'longitude'])
    return geopy.distance.distance(coord1, coord2).km

calculate_distance(0, 1)

In [ ]:
#CLIENT_LOCATIONS

assignments = pandas.read_csv("usa_va_charlottesville_city_adult_activity_location_assignment_week.csv")
HOME_SHIFT=1000000000

#print(df_residence[df_residence['lid']==1018209+HOME_SHIFT])
home_locs = set(pandas.read_csv("usa_va_charlottesville_city_residence_locations.csv").rlid)

assignments['home'] = assignments['lid'].apply(lambda x: True if x in home_locs else False)
assignments.loc[assignments.home, "lid"] += HOME_SHIFT
#print(assignments.head())
#assignments.loc[assignments.activity_type == 1, "lid"] += HOME_SHIFT
#print(assignments[assignments['lid']==1229162])

assignments = assignments.groupby(['pid'])['lid'].apply(set).reset_index(name = 'lid')

def filter(x):
    return_list = []
    for i in x:
        if (i>HOME_SHIFT):
            return_list.insert(0, location_dict.loc[i, 'index'])
        else:
            return_list.append(location_dict.loc[i, 'index'])
    return return_list
    
assignments['lid'] = assignments['lid'].apply(lambda x: filter(x))

for loc in assignments.loc[0, 'lid']:
    print(locations.loc[loc, 'lid'])

In [ ]:
import pandas as pd

HOME_SHIFT=1000000000

def create_location_data():
    df_activity = pd.read_csv("usa_va_charlottesville_city_activity_locations.csv").rename({"alid": "lid"}, axis = 'columns')
    df_residence = pd.read_csv("usa_va_charlottesville_city_residence_locations.csv")
    
    df_residence['rlid'] = df_residence['rlid'].apply(lambda x: x+HOME_SHIFT)
    df_residence = df_residence.rename({"rlid": "lid"}, axis = 'columns')
    locations = pd.concat([df_activity[['lid', 'longitude', 'latitude']], df_residence[['lid', 'longitude', 'latitude']]]).reset_index(drop = True)
    
    client_locations = pd.read_csv("usa_va_charlottesville_city_adult_activity_location_assignment_week.csv")
    home_coords = client_locations.loc[client_locations.activity_type==1,['latitude', 'longitude']]
    home_coords = set(home_coords[['latitude', 'longitude']].apply(tuple, axis=1).tolist())
    client_locations['home'] = client_locations.apply(lambda x: True if (x.loc['latitude'], x.loc['longitude']) in home_coords else False, axis = 1)
    client_locations.loc[client_locations.home == True, 'lid'] += HOME_SHIFT
    
    location_dict = locations[['lid']]
    location_dict.reset_index(inplace=True)
    location_dict = location_dict.set_index('lid')
    
    #Popularity of locations, which locations are visited by which individuals
    # TODO: Merge with locations data?
    assignments = client_locations.copy()
    assignments = assignments.groupby(['lid'])['pid'].apply(set).reset_index(name = 'pid')
    assignments['lid'] = assignments['lid'].apply(lambda x: location_dict.loc[x, 'index'])
    assignments['activity'] = assignments['pid'].apply(lambda x: len(x))
    #locations.merge(assignments, on = 'lid')
    assignments = assignments.sort_values(by = 'activity', ascending = False)
    
    client_locations = client_locations.groupby(['pid'])['lid'].apply(set).reset_index(name = 'lid')
    
    #Replace lid with the index of the lid in locations
    def filter(x):
        return_list = []
        for i in x:
            #Insert home locations at the front of the list
            if (i>HOME_SHIFT):
                return_list.insert(0, location_dict.loc[i, 'index'])
            else:
                return_list.append(location_dict.loc[i, 'index'])
        return return_list

    client_locations['lid'] = client_locations['lid'].apply(lambda x: filter(x))

    return locations, assignments, client_locations

LOCATIONS, LOCATION_ASSIGNMENTS, CLIENT_LOCATIONS = create_location_data()

In [ ]:
potential_facility_locations = set(LOCATION_ASSIGNMENTS.iloc[range(s)].lid)

#Remove homes from the client_location lists
#TODO: Perhaps create mapping for the indices of people before exclusion and after?
client_locations_excluded = []
for person in CLIENT_LOCATIONS.lid:
    new_list = list(set(person[1:]).intersection(potential_facility_locations))
    if len(new_list)>0:
        client_locations_excluded.append(new_list)


In [ ]:
LOCATIONS, LOCATION_ASSIGNMENTS, CLIENT_LOCATIONS = create_location_data()

In [ ]:
loc_dict = {'lid':[5, 10, 15], 'latitude':[0, 1, -1], 'longitude':[0, -1, 1]}
loc_assign_dict = {'lid':[0, 1, 2], 'activity':[2, 5, 10]}
client_loc_dict = {'pid':[0, 1, 2], 'lid':[[0, 1], [0, 2], [0, 1, 2]]}
LOCATIONS, LOCATION_ASSIGNMENTS, CLIENT_LOCATIONS = pd.DataFrame.from_dict(loc_dict), pd.DataFrame.from_dict(loc_assign_dict), pd.DataFrame.from_dict(client_loc_dict)

print(LOCATIONS.head())
print(LOCATION_ASSIGNMENTS.head())
print(CLIENT_LOCATIONS.head())

In [60]:
from utils import *

def test_function(k:int):
    print(CLIENT_LOCATIONS.head())
    
    '''print("------------IND ROUNDING------------------")
    X_ind, Y_ind = independent_LP(k)
    format_location_output(X_ind, Y_ind)
    print("Recalculated Objective Value: \t" + str(calculate_objective(Y_ind)))'''
    
    '''print("------------INTEGER LP--------------------")
    X_int, Y_int = integer_LP(k)
    format_location_output(X_int, Y_int)
    print("Recalculated Objective Value: \t" + str(calculate_objective(Y_int)))
    
    print("------------DEP ROUNDING-----------------")
    X_dep, Y_dep = dependent_LP(k)
    format_location_output(X_dep, Y_dep)
    print("Recalculated Objective Value: \t" + str(calculate_objective(Y_dep)))
    '''
    '''print("----------------FPT----------------------")
    X_fpt, Y_fpt = fpt(k, 5)
    format_location_output(X_fpt, Y_fpt)
    print("Recalculated Objective Value: \t" + str(calculate_objective(Y_fpt)))'''
    
    print("----------Center of Homes----------------")
    X_home, Y_home = center_of_homes(k)
    format_location_output(X_home, Y_home)
    print("Recalculated Objective Value: \t" + str(calculate_objective(Y_home)))
    
    '''print("----------Center of Centers--------------")
    X_center, Y_center = center_of_centers(k)
    format_location_output(X_center, Y_center)
    print("Recalculated Objective Value: \t" + str(calculate_objective(Y_center)))'''

test_function(20)

       pid                                                lid
0  5586585  [12796, 2934, 4586, 1313, 3456, 915, 3246, 294...
1  5586586  [12796, 8293, 4404, 1681, 8025, 527, 5924, 332...
2  5586591  [12892, 5162, 5143, 2738, 4481, 7223, 5697, 51...
3  5586592  [12892, 4887, 8984, 8264, 4233, 8072, 2825, 62...
4  5586593  [12893, 3557, 8648, 7199, 3152, 6727, 8025, 86...
----------Center of Homes----------------
{4481, 8067, 6283, 4887, 8984, 3996, 8994, 5162, 8890, 4285, 5188, 8266, 5842, 6866, 8057, 8559, 8182, 9081, 5243, 8957}
20
Facilities Opened: 	[4481, 8067, 6283, 4887, 8984, 3996, 8994, 5162, 8890, 4285, 5188, 8266, 5842, 6866, 9081, 8559, 8182, 8057, 5243, 8957]
Client Assignment: 	[(12796, 8266), (12796, 8266), (12892, 8266), (12892, 8266), (12893, 8266), (12983, 8266), (13023, 8984), (13023, 8984), (13023, 8984), (13081, 8266), (13081, 8266), (13082, 8266), (13082, 8266), (13165, 8266), (13165, 8266), (13165, 8266), (13166, 8266), (13168, 8266), (13212, 8266), (13212, 8266), 

In [ ]:
print(LOCATION_ASSIGNMENTS.head())

lid = LOCATION_ASSIGNMENTS.loc[0, 'lid']
boo = True
print(lid)
#print(LOCATIONS_ASSIGNMENTS.lid.to_list())
for p in LOCATION_ASSIGNMENTS.loc[0, 'pid']:
    print(p)
    #print((CLIENT_LOCATIONS.loc[CLIENT_LOCATIONS.pid==p].lid).tolist())
    if not lid in (CLIENT_LOCATIONS.loc[CLIENT_LOCATIONS.pid==p].lid).tolist().pop():
        print("FALSE")

In [ ]:
potential_facility_locations = LOCATION_ASSIGNMENTS.iloc[range(20)].lid.tolist()
#print(potential_facility_locations)
homes = [locs[0] for locs in CLIENT_LOCATIONS.lid]

locations = [i for i in potential_facility_locations if i not in homes]
print(locations)
#if LOCATIONS.loc[h, 'lid']:
#  print("FALSE")

clients = homes

#for c in clients:
    #print(min([calculate_distance(c, f) for f in locations]))
#max_min_dist = max([min([calculate_distance(c, f) for f in locations]) for c in clients])
#print(max_min_dist)

In [17]:
HOME_SHIFT=1000000000

def create_location_data2():
    """
    RETURNS
    --------
    locations: pd.DataFrame
        Contains lid (location id), latitude, longitude
        Index is set as lid
    assignments: pd.DataFrame
        Contains lid_index (the index mapping to each lid from locations), a set of pid (personal ids) visiting those locations, 
        and activity (the size of the set) as a measure of popularity
        lid_index is simply titled as 'lid' in the dataframe
    client_locations: pd.DataFrame
        Contains pid, set of lid_index visited by each pid with the home location as the first element
        lid_index is simply titled as 'lid' in the dataframe
    """
    
    #Read in both the activity and residence locations
    df_activity = pd.read_csv("usa_va_charlottesville_city_activity_locations.csv").rename({"alid": "lid"}, axis = 'columns')
    df_residence = pd.read_csv("usa_va_charlottesville_city_residence_locations.csv")
    
    #Shift the residence lid
    df_residence['rlid'] = df_residence['rlid'].apply(lambda x: x+HOME_SHIFT)
    df_residence = df_residence.rename({"rlid": "lid"}, axis = 'columns')
    locations = pd.concat([df_activity[['lid', 'longitude', 'latitude']], df_residence[['lid', 'longitude', 'latitude']]]).reset_index(drop = True)
    locations = locations.set_index('lid')
    
    #Read in the client visited locations data (adults only for now)
    client_locations = pd.read_csv("usa_va_charlottesville_city_adult_activity_location_assignment_week.csv")
    
    #Get the coordinates of all the residential locations
    home_coords = set(df_residence[['latitude', 'longitude']].apply(tuple, axis=1).tolist())
    
    #Shift lids for residential areas
    client_locations['home'] = client_locations.apply(lambda x: True if (x.loc['latitude'], x.loc['longitude']) in home_coords else False, axis = 1)
    client_locations.loc[client_locations.home == True, 'lid'] += HOME_SHIFT
    
    #Assess popularity of locations, which locations are visited by which individuals
    assignments = client_locations.copy()
    assignments = assignments.groupby(['lid'])['pid'].apply(set).reset_index(name = 'pid')
    assignments['latitude'] = assignments['lid'].apply(lambda x: locations.loc[x, 'latitude'])
    assignments['longitude'] = assignments['lid'].apply(lambda x: locations.loc[x, 'longitude'])
    assignments['activity'] = assignments['pid'].apply(lambda x: len(x))
    assignments = assignments.sort_values(by = 'activity', ascending = False).reset_index(drop = True)
    assignments = assignments.set_index('lid')
    
    client_locations = client_locations.groupby(['pid'])['lid'].apply(set).reset_index(name = 'lid')

    #Replace lid with the index of the lid in locations
    def filter(x):
        return_list = []
        for i in x:
            #Insert home locations at the front of the list
            if (i>HOME_SHIFT):
                return_list.insert(0, i)
            else:
                return_list.append(i)
        return return_list

    client_locations['lid'] = client_locations['lid'].apply(lambda x: filter(x))

    return assignments, client_locations

LOCATION_ASSIGNMENTS, CLIENT_LOCATIONS = create_location_data2()

                                                          pid   latitude  \
lid                                                                        
82889       {5603328, 5603334, 5603337, 5603343, 5603347, ...  38.059300   
91203       {5603332, 5603335, 5603352, 5603361, 5603366, ...  38.030710   
130369      {5603330, 5603334, 5603362, 5603373, 5603381, ...  38.031040   
94144       {5603348, 5603365, 5603383, 5603385, 5603389, ...  38.062380   
127361      {5603345, 5603346, 5603347, 5603372, 5603373, ...  38.039860   
...                                                       ...        ...   
131493                                              {5600213}  39.067690   
131511                                              {5623992}  39.075320   
68489                                               {5589026}  37.645840   
68473                                               {5602365}  38.838170   
1002732592                                          {5602693}  38.012541   

           

In [36]:
#print(LOCATIONS.head())
#print(LOCATION_ASSIGNMENTS.loc[15])
print(set(LOCATION_ASSIGNMENTS.iloc[:10].index))
print(CLIENT_LOCATIONS.lid[0])

{94144, 130369, 127361, 91203, 134340, 82889, 105075, 100662, 94936, 134459}
[1001018209, 67712, 88322, 25738, 76430, 18451, 73494, 67867, 4268, 107194, 72382, 94144, 86726, 82246, 20295, 122200, 75609, 64858, 81115, 7390]
